In [1]:
from azure.cosmosdb.table.tableservice import TableService
from azure.cosmosdb.table.models import Entity

import azure_db_credentials

import pandas as pd

In [2]:
storage_account_name = azure_db_credentials.storage_account_name
sas_token = azure_db_credentials.sas_token

In [3]:
def query_azure_db(filterQuery, selectQuery, tableName, rowKeyColumnName):
    
    ## Setup the table service
    table_service = TableService(account_name=storage_account_name, sas_token=sas_token)
    
    ## Get the data from the Storage Account
    transactions = list(table_service.query_entities(tableName, filter=filterQuery, select=selectQuery))
    
    ## Store data in a Pandas dataframe in a neat way
    df = pd.DataFrame(transactions)
    df.rename(columns={'RowKey': rowKeyColumnName}, inplace=True)
    df.set_index(rowKeyColumnName, inplace=True)
    df.drop(['etag'], axis=1, inplace=True)

    return df

In [4]:
def get_data_from_historical_transactions():
    filterQuery = "deployment eq 'production' and stopTime ne '' and chg3phase ne '' and maxPowerDetermined eq true"
    selectQuery = 'RowKey, chargingStationId, chg3phase, connectorId, idTag, energy, finished, lastSeen, curPower, maxPowerSeen, meterStart, meterStop, startTime, stopTime'
    tableName = "HistoricalTransactions"
    rowKeyColumnName = "transactionId"
    
    return query_azure_db(filterQuery, selectQuery, tableName, rowKeyColumnName)

In [5]:
def get_power_data_from_meter_values(partitionKey):
    filterQuery = "PartitionKey eq '" + str(partitionKey) + "' and measurand eq 'Power.Active.Import'"
    selectQuery = 'PartitionKey, RowKey, chargingStationId, connectorId, timestampMeasurement, value'
    tableName = "MeterValues"
    rowKeyColumnName = "meterValueId"
    
    return query_azure_db(filterQuery, selectQuery, tableName, rowKeyColumnName)

In [6]:
data = get_data_from_historical_transactions()
print(data.head())

              chargingStationId  chg3phase  connectorId  curPower  energy  \
transactionId                                                               
4120714                  ALFEN1      False            2       0.0   11570   
8590630                  BLUEC2       True            1       0.0   11590   
9330752                  ALFEN1      False            1       0.0   10800   
1150705                  ALFEN1      False            1       0.0   17180   
2580630                  ALFEN1      False            2       0.0   11200   

               finished           idTag              lastSeen  maxPowerSeen  \
transactionId                                                                 
4120714            True  04974FFAB63780  2018-08-08T14:31:30Z        3414.0   
8590630            True  043661B2B63780  2018-08-08T15:48:06Z       21558.0   
9330752            True  044627B2B63780  2018-08-08T14:52:45Z        6618.0   
1150705            True  044D79B2B63780  2018-08-13T15:23:08Z    

In [7]:
data = get_power_data_from_meter_values(4120714)
print(data.head())

                                     PartitionKey chargingStationId  \
meterValueId                                                          
001e42a7-0c5e-47c6-a5e0-9f4ca7ba3087      4120714            ALFEN1   
0021062a-04da-4a3a-a1fe-92b057f76e8c      4120714            ALFEN1   
00314cab-ff00-4d8f-9563-48d0cbde50a8      4120714            ALFEN1   
0050db6a-e85f-4270-b383-4a63062d4a77      4120714            ALFEN1   
0060eaa6-880b-4152-bba9-fa8fbde2e383      4120714            ALFEN1   

                                     connectorId  timestampMeasurement  \
meterValueId                                                             
001e42a7-0c5e-47c6-a5e0-9f4ca7ba3087           2  2018-08-08T07:45:03Z   
0021062a-04da-4a3a-a1fe-92b057f76e8c           2  2018-08-08T13:04:43Z   
00314cab-ff00-4d8f-9563-48d0cbde50a8           2  2018-08-08T09:47:03Z   
0050db6a-e85f-4270-b383-4a63062d4a77           2  2018-08-08T09:05:43Z   
0060eaa6-880b-4152-bba9-fa8fbde2e383           2  2018-08-